In [1]:
import tensorflow as tf
import keras 
from keras.layers import Conv2D , MaxPooling2D
from keras import backend as k
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Đọc dữ liệu từ tệp CSV
data = pd.read_csv(r"C:\Users\MinhThw\OneDrive\Desktop\Autism data ADS\Autism Dataset for Toddlers.csv", sep=";")


# Đặt tên cột cho DataFrame
#raw_data.columns = ['Qchat-10-Score', 'Sex', 'Ethnicity', 'Jaundice', 'Family_mem_with_ASD', 'Who completed the test', 'Class/ASD Traits']
raw_data=data.iloc[:, [12,13,14,15,16,18]]

print (raw_data)

# Biến đổi dữ liệu
result = pd.concat([
    pd.get_dummies(raw_data['Jaundice']).drop(['no'], axis=1).rename(columns={'yes': 'Jaundice'}),
    pd.get_dummies(raw_data['Sex']).drop(['m'], axis=1).rename(columns={'f': 'Sex'}),
    pd.get_dummies(raw_data['Qchat-10-Score'], prefix='Qchat-10-Score_'),
    pd.get_dummies(raw_data['Family_mem_with_ASD']).drop(['no'], axis=1).rename(columns={'yes': 'Family_mem_with_ASD'}),
    pd.get_dummies(raw_data['Ethnicity']),
    #pd.get_dummies(raw_data['Who completed the test']).replace({'family member': 1, 'other': 0}).rename(columns={'family member': 'Who completed the test'}),
    pd.get_dummies(raw_data['Class/ASD Traits']).drop(['No'], axis=1).rename(columns={'Yes': 'Class/ASD Traits'})
], axis=1)

# In kết quả
print((result).astype(int))



      Qchat-10-Score Sex       Ethnicity Jaundice Family_mem_with_ASD  \
0                  3   f  middle eastern      yes                  no   
1                  4   m  White European      yes                  no   
2                  4   m  middle eastern      yes                  no   
3                 10   m        Hispanic       no                  no   
4                  9   f  White European       no                 yes   
...              ...  ..             ...      ...                 ...   
1049               1   f  White European       no                 yes   
1050               5   m           black      yes                  no   
1051               9   m  middle eastern      yes                  no   
1052               3   m  White European       no                 yes   
1053               6   m           asian      yes                 yes   

     Class/ASD Traits  
0                  No  
1                 Yes  
2                 Yes  
3                 Yes  
4  

In [2]:
result.columns.values

array(['Jaundice', 'Sex', 'Qchat-10-Score__0', 'Qchat-10-Score__1',
       'Qchat-10-Score__2', 'Qchat-10-Score__3', 'Qchat-10-Score__4',
       'Qchat-10-Score__5', 'Qchat-10-Score__6', 'Qchat-10-Score__7',
       'Qchat-10-Score__8', 'Qchat-10-Score__9', 'Qchat-10-Score__10',
       'Family_mem_with_ASD', 'Hispanic', 'Latino', 'Native Indian',
       'Others', 'Pacifica', 'White European', 'asian', 'black',
       'middle eastern', 'mixed', 'south asian', 'Class/ASD Traits'],
      dtype=object)

In [3]:

#X = result.to_numpy()
  

# Chia thành tập huấn luyện và tập kiểm tra
data_row_dim = 1
data_col_dim = 26
X = result.drop(['Class/ASD Traits'], axis = 1).to_numpy()
print(X.shape)
#X = X.values.reshape(-1, data_row_dim, data_col_dim)
#print(X.shape)   
y_temp = result['Class/ASD Traits']
# Chuyển đổi kiểu dữ liệu của y thành số thực
y = y_temp.dropna().astype('float32').to_numpy()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#X_train = X_train.reshape(X_train.shape[0],data_row_dim,data_col_dim,1)
#X_test = X_test.reshape(X_test.shape[0],data_row_dim,data_col_dim,1)
input_shape = (data_row_dim,data_col_dim,1)

print(X_train.shape , y_train.shape)
print(X_test.shape , y_test.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#print('X_train.shape', X_test.shape)
print(X_train.shape[0],'Train Sample')
print(X_test.shape[0],'Test Sample')

#Tạo Model

model = Sequential()
model.add(Dense(units=32,activation='relu'))
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=1))

# Compile model
model.compile( loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"]) # categorical_crossentropy'
#model.compile(loss = 'mse',optimizer = 'adam')

# Model training
model.fit(X_train,y_train , batch_size=128,epochs=15,verbose=1,validation_data=(X_test,y_test))

#model.fit(X_train,y_train, batch_size=128, validation_data=(X_test,y_test), epochs=40, verbose=1)
print("The Model has Successfully Trained")



(1054, 25)
(737, 25) (737,)
(317, 25) (317,)
(737, 25) (737,)
(317, 25) (317,)
737 Train Sample
317 Test Sample
Epoch 1/15
6/6 [==============================] - 1s 47ms/step - loss: 3.9826 - accuracy: 0.3012 - val_loss: 0.9761 - val_accuracy: 0.3344
Epoch 2/15
6/6 [==============================] - 0s 9ms/step - loss: 0.8452 - accuracy: 0.3826 - val_loss: 0.6755 - val_accuracy: 0.5521
Epoch 3/15
6/6 [==============================] - 0s 8ms/step - loss: 0.7348 - accuracy: 0.6201 - val_loss: 0.7561 - val_accuracy: 0.6751
Epoch 4/15
6/6 [==============================] - 0s 9ms/step - loss: 0.7723 - accuracy: 0.7123 - val_loss: 0.7436 - val_accuracy: 0.7319
Epoch 5/15
6/6 [==============================] - 0s 8ms/step - loss: 0.7081 - accuracy: 0.7517 - val_loss: 0.5658 - val_accuracy: 0.7697
Epoch 6/15
6/6 [==============================] - 0s 8ms/step - loss: 0.5572 - accuracy: 0.7883 - val_loss: 0.4125 - val_accuracy: 0.7950
Epoch 7/15
6/6 [==============================] - 0s 8ms/st

In [ ]:
model.history.history.keys()

model_loss = pd.DataFrame(model.history.history) # Bai toan regression ko co metric "accuracy"
model_loss['loss']

plt.figure()
plt.plot(model_loss['loss'], label= 'loss')
plt.plot(model_loss['val_loss'], label= 'val_loss')
plt.legend(['loss', 'val_loss'])

predictions = model.predict(X_test)
predictions


In [ ]:

# y_pred = tf.argmax(predictions, axis=-1)

from sklearn.metrics import confusion_matrix, classification_report

# Model Saving
model.save('Book1.model')
print('Saving the model as Autism_Dataset_for_Toddlers.h5')

# Save and load model  

new_model=load_model('Book1.model')
predictions = new_model.predict(X_test)
print(predictions)
y_pred = tf.argmax(predictions, axis=-1)
print(y_pred)

In [ ]:
#f1_score = f1_score(y_test,y_pred,average='macro') 
#calculate the accuracy and store it in a
#accuracy_score = accuracy_score(y_test,y_pred)

#print(f'f1_score: {f1_score}')
#print(f'accuracy_score: {accuracy_score}')